# Load Test on MyAdroit (GPU nodes in the cloud)

In [ ]:
%%time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR

In [ ]:
!nvidia-smi

In [ ]:
import os
from time import time
from datetime import datetime
start_dt = datetime.now()
start = time()

use_cuda = torch.cuda.is_available()

torch.manual_seed(42)
device = torch.device("cuda") if use_cuda else torch.device("cpu")

train_kwargs = {'batch_size': 64}
test_kwargs  = {'batch_size': 192}
if use_cuda:
    cuda_kwargs = {'num_workers': 4, 'pin_memory': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])
print("Creating data loaders ... ", end="", flush=True)
dataset1 = datasets.ImageFolder(root="/home/jdh4/cats_vs_dogs/training_set/", transform=transform)
dataset2 = datasets.ImageFolder(root="/home/jdh4/cats_vs_dogs/test_set/", transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, shuffle=True, **train_kwargs)
test_loader  = torch.utils.data.DataLoader(dataset2, shuffle=True, **test_kwargs)
print("done.", flush=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, 1)
        self.conv2 = nn.Conv2d(64, 128, 3, 1)
        self.conv3 = nn.Conv2d(128, 32, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout3 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(28800, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout3(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print(f"epoch {epoch}, batch {batch_idx}")

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

epochs = 4
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

print(f"{round(time() - start)} seconds, timestamp={start_dt}, {os.environ['USER']}, cuda={use_cuda}")